In [ ]:
def ff_gen_template_video(clip_bg_fp,final_template_video_fp):
    import subprocess,time
    st=time.time()
    subprocess.run(['ffmpeg', '-i', clip_bg_fp, '-vf', 'scale=1080:1920', '-y', 'content/resized_image.png'])
    subprocess.run(['ffmpeg','-y','-loop','1','-i','content/resized_image.png','-i','content/video_template_with_mask.mov','-filter_complex','[0:v][1:v]overlay=shortest=1',final_template_video_fp])
    subprocess.run(['rm', 'resized_image.png'])
    print(f'gen_template run time:{time.time()-st}')

In [ ]:
def sd_gen_template_video(prompt,final_template_video_fp):
    import requests,base64
    url = "http://sd.insnail.com"
    option_resp = requests.get(f'{url}/sdapi/v1/options')
    if "realisticStockPhoto" not in option_resp.json()['sd_model_checkpoint']:
        option_payload = {
            "sd_model_checkpoint": "realisticStockPhoto_v30SD15",
        }
        option_response = requests.post(url=f'{url}/sdapi/v1/options', json=option_payload)
        print(f'{option_response.json()}')
    payload = {
    "prompt": f"{prompt},<lora:Anchemix realistic house SDXL V1-000010:0.7>",
    "steps": 20,
    "width": 720,
    "height": 1280,
    "refiner_checkpoint": "realisticStockPhoto_v30SD15",
    "refiner_switch_at":0.8
    }
    response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)
    print(f'bgimage_generated:{response}')
    with open("temp/output.png", 'wb') as f:
        f.write(base64.b64decode(response.json()['images'][0]))
    ff_gen_template_video('temp/output.png',final_template_video_fp)

In [ ]:
prompt= "Anchemix house,Anchemix realistic,no humans,scenery,table,indoors,chair,plant,couch,window,pillow,door,shadow"
sd_gen_template_video(prompt,'temp/ff_template_gen.mp4')